In [1]:
import os
import os.path
import glob
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold                                                                                                                       
from sklearn.metrics import confusion_matrix,accuracy_score
from sklearn import svm
from sklearn.model_selection import GridSearchCV
import sklearn_evaluation.plot as skplot
from sklearn.model_selection import learning_curve

import matplotlib.pyplot as plt

import numpy as np
np.random.seed(1)

from keras.preprocessing.image import img_to_array
from keras.utils import np_utils
from keras.preprocessing import image
from keras.applications.xception import Xception,preprocess_input

os.environ['CUDA_VISIBLE_DEVICES'] = '1'

Using TensorFlow backend.


In [2]:
imagedir = "Datasets/Detection/Byteplots"

In [3]:
cur_dir = os.getcwd()
os.chdir(imagedir)  # the parent folder with sub-folders

# Get number of samples per family
list_fams = sorted(os.listdir(os.getcwd()), key=str.lower)  # vector of strings with family names
no_imgs = []  # No. of samples per family
for i in range(len(list_fams)):
    os.chdir(list_fams[i])
    len1 = len(glob.glob('*.png'))  # assuming the images are stored as 'png'
    no_imgs.append(len1)
    os.chdir('..')
num_samples = np.sum(no_imgs)  # total number of all samples

# Compute the labels
y = np.zeros(num_samples)
pos = 0
label = 0
for i in no_imgs:
    print ("Label:%2d\tFamily: %15s\tNumber of images: %d" % (label, list_fams[label], i))
    for j in range(i):
        y[pos] = label
        pos += 1
    label += 1
num_classes = label

# Compute the features
width, height,channels = (224,224,3)
X = np.zeros((num_samples, width, height, channels))
cnt = 0
list_paths = [] # List of image paths
print("Processing images ...")
for i in range(len(list_fams)):
    for img_file in glob.glob(list_fams[i]+'/*.png'):
        #print("[%d] Processing image: %s" % (cnt, img_file))
        list_paths.append(os.path.join(os.getcwd(),img_file))
        img = image.load_img(img_file, target_size=(224, 224))
        x = image.img_to_array(img)
        x = np.expand_dims(x, axis=0)
        x = preprocess_input(x)
        X[cnt] = x
        cnt += 1
print("Images processed: %d" %(cnt))

os.chdir(cur_dir)

Label: 0	Family:          Banker	Number of images: 1986
Label: 1	Family:          WinExe	Number of images: 1945
Processing images ...


/usr/local/lib/python2.7/dist-packages/PIL/Image.py:2371: DecompressionBombWarning: Image size (103265280 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  DecompressionBombWarning)
/usr/local/lib/python2.7/dist-packages/PIL/Image.py:2371: DecompressionBombWarning: Image size (110601216 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  DecompressionBombWarning)


Images processed: 3931


In [4]:
X.shape

(3931, 224, 224, 3)

In [5]:
# Creating base_model
image_shape = (224, 224, 3)                                                                                                                                                                                                                                                                                            
base_model = Xception(weights='imagenet', input_shape=image_shape, include_top=False, pooling='avg')

In [6]:
print("Extracting bottleneck features from convolutional layers ...")
bnfeatures = base_model.predict(X)

Extracting bottleneck features from convolutional layers ...


In [7]:
bnfeatures.shape

(3931, 2048)

In [8]:
# Create stratified k-fold subsets                                                                                                                                        
kfold = 5  # no. of folds                                                                 
skf = StratifiedKFold(kfold, shuffle=True,random_state=1)

In [9]:
# Training top_model
C_range = np.logspace(-2, 10, 13)
param_grid = dict(C=C_range)

top_model = GridSearchCV(svm.LinearSVC(), param_grid=param_grid, cv=skf, verbose=10, n_jobs=-1)
top_model.fit(bnfeatures,y)  # Training 

Fitting 5 folds for each of 13 candidates, totalling 65 fits
[CV] C=0.01 ..........................................................
[CV] C=0.01 ..........................................................
[CV] C=0.01 ..........................................................
[CV] C=0.01 ..........................................................
[CV] C=0.01 ..........................................................
[CV] C=0.1 ...........................................................
[CV] C=0.1 ...........................................................
[CV] C=0.1 ...........................................................
[CV] C=0.1 ...........................................................
[CV] C=0.1 ...........................................................
[CV] C=1.0 ...........................................................
[CV] C=1.0 ...........................................................
[CV] C=1.0 ...........................................................
[CV] C=1.0 .....

[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    4.7s


[CV] ........................... C=0.01, score=0.940204, total=   4.3s
[CV] C=100.0 .........................................................
[CV] C=100.0 .........................................................
[CV] C=1000.0 ........................................................
[CV] ........................... C=0.01, score=0.945362, total=   5.1s
[CV] C=1000.0 ........................................................
[CV] C=1000.0 ........................................................
[CV] ........................... C=0.01, score=0.933842, total=   5.1s
[CV] C=1000.0 ........................................................
[CV] ............................ C=0.1, score=0.966921, total=  15.1s
[CV] C=1000.0 ........................................................
[CV] ............................ C=1.0, score=0.937659, total=  14.7s
[CV] C=10000.0 .......................................................
[CV] ........................... C=10.0, score=0.949174, total=  15.3s
[CV] C

[Parallel(n_jobs=-1)]: Done  13 tasks      | elapsed:   24.9s


[CV] ......................... C=1000.0, score=0.927481, total=  18.9s
[CV] C=100000.0 ......................................................
[CV] ............................ C=0.1, score=0.940204, total=  23.8s
[CV] C=100000.0 ......................................................
[CV] ......................... C=1000.0, score=0.942748, total=  10.0s
[CV] C=100000.0 ......................................................
[CV] ........................ C=10000.0, score=0.927481, total=   8.7s
[CV] C=1000000.0 .....................................................
[CV] ........................ C=10000.0, score=0.947903, total=  10.3s
[CV] C=1000000.0 .....................................................
[CV] ........................ C=10000.0, score=0.952926, total=   8.7s
[CV] C=1000000.0 .....................................................
[CV] ........................... C=10.0, score=0.926209, total=  25.9s
[CV] C=1000000.0 .....................................................
[CV] .

...

[CV] .................... C=100000000.0, score=0.927481, total=  15.3s
[CV] .................. C=10000000000.0, score=0.926209, total=   8.8s
[CV] ..................... C=10000000.0, score=0.952926, total=  17.1s
[CV] .................. C=10000000000.0, score=0.946633, total=   9.9s
[CV] ................... C=1000000000.0, score=0.924936, total=  14.7s
[CV] .................... C=100000000.0, score=0.947903, total=  16.4s
[CV] .................... C=100000000.0, score=0.952926, total=  15.8s


[Parallel(n_jobs=-1)]: Done  60 out of  65 | elapsed:   47.8s remaining:    4.0s


[CV] ................... C=1000000000.0, score=0.952926, total=  14.4s
[CV] .................. C=10000000000.0, score=0.952926, total=   8.3s
[CV] .................. C=10000000000.0, score=0.928753, total=   8.7s
[CV] .................. C=10000000000.0, score=0.942748, total=   8.4s


[Parallel(n_jobs=-1)]: Done  65 out of  65 | elapsed:   49.8s finished


GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=1, shuffle=True),
       error_score='raise',
       estimator=LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0),
       fit_params={}, iid=True, n_jobs=-1,
       param_grid={'C': array([  1.00000e-02,   1.00000e-01,   1.00000e+00,   1.00000e+01,
         1.00000e+02,   1.00000e+03,   1.00000e+04,   1.00000e+05,
         1.00000e+06,   1.00000e+07,   1.00000e+08,   1.00000e+09,
         1.00000e+10])},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=10)

In [10]:
top_model.cv_results_

{'mean_fit_time': array([  4.19867325,  21.18969688,  25.6548193 ,  24.14653616,
         22.43516884,  20.80662475,  10.96411905,  14.85016942,
         11.89426675,  14.83085141,  14.67785382,  11.8375062 ,   8.81231699]),
 'mean_score_time': array([ 0.0938262 ,  0.10314813,  0.07468834,  0.07859397,  0.10499978,
         0.06364498,  0.07921801,  0.03368697,  0.03297119,  0.06931686,
         0.03924241,  0.03837314,  0.00568819]),
 'mean_test_score': array([ 0.94530654,  0.95141185,  0.94479776,  0.94072755,  0.93996439,
         0.93996439,  0.93996439,  0.93996439,  0.93971   ,  0.94021877,
         0.93996439,  0.93920122,  0.93945561]),
 'mean_train_score': array([ 0.97201724,  0.99274995,  1.        ,  1.        ,  1.        ,
         1.        ,  1.        ,  1.        ,  0.99993639,  0.99993639,
         1.        ,  0.99993641,  1.        ]),
 'param_C': masked_array(data = [0.01 0.10000000000000001 1.0 10.0 100.0 1000.0 10000.0 100000.0 1000000.0
  10000000.0 100000000.0 

In [11]:
top_model.grid_scores_

/usr/local/lib/python2.7/dist-packages/sklearn/model_selection/_search.py:667: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


[mean: 0.94531, std: 0.01027, params: {'C': 0.01},
 mean: 0.95141, std: 0.01094, params: {'C': 0.10000000000000001},
 mean: 0.94480, std: 0.00988, params: {'C': 1.0},
 mean: 0.94073, std: 0.01085, params: {'C': 10.0},
 mean: 0.93996, std: 0.01026, params: {'C': 100.0},
 mean: 0.93996, std: 0.01020, params: {'C': 1000.0},
 mean: 0.93996, std: 0.01020, params: {'C': 10000.0},
 mean: 0.93996, std: 0.01020, params: {'C': 100000.0},
 mean: 0.93971, std: 0.01106, params: {'C': 1000000.0},
 mean: 0.94022, std: 0.01041, params: {'C': 10000000.0},
 mean: 0.93996, std: 0.01020, params: {'C': 100000000.0},
 mean: 0.93920, std: 0.01111, params: {'C': 1000000000.0},
 mean: 0.93946, std: 0.01033, params: {'C': 10000000000.0}]

In [12]:
print("Best parameters: %s - score: %0.4f" % (top_model.best_params_, top_model.best_score_))

Best parameters: {'C': 0.10000000000000001} - score: 0.9514
